In [1]:
import numpy as np
import pickle
import tensorflow as tf
import time
import matplotlib.pyplot as plt
import math
import model5
import scipy
from scipy import ndimage
from sklearn.model_selection import train_test_split

/usr/lib/python3/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])

In [3]:
def parse_image(image, y_size, x_size, z_size):
    image = image[0:x_size, 0:y_size, 0:z_size]
    num_patches_per_x = image.shape[0] // 156
    num_patches_per_y = image.shape[1] // 156
    num_patches_per_z = image.shape[2] // 100
    x_size_crop = num_patches_per_x * 156
    y_size_crop = num_patches_per_y * 156
    z_size_crop = num_patches_per_z * 100
    image = image[0:x_size_crop, 0:y_size_crop, 0:z_size_crop]
    patches = image.reshape(num_patches_per_x, 156,
                            num_patches_per_y, 156, num_patches_per_z, 100)
    patches = patches.transpose(0,2,4, 1,3,5)
    patches = patches.reshape(num_patches_per_x * num_patches_per_y * num_patches_per_z,
                              156, 156, 100)
    return patches

In [4]:
def sep_image(image, x_size, y_size, z_size):
    image = image[0:x_size, 0:y_size, 0:z_size]
    image1=image[:,62:217,:]
    image2=image[:,0:156,:]
    image3=image[:,124:280,:]
    image4=image[:,:,284:440]
    image1=image1.transpose(2,1,0)
    image2=image2.transpose(2,1,0)
    image3=image3.transpose(2,1,0)
    image4=image4.transpose(2,1,0)
    data1=parse_image(image1,156,440,100)
    data2=parse_image(image2,156,440,100)
    data3=parse_image(image3,156,440,100)
    data4=parse_image(image4,280,156,100)
    return data1,data2,data3,data4

In [5]:
a=np.zeros((3,4))
a.shape
a[1:3,1:4].shape

(2, 3)

In [6]:
import pickle
with open('chan.p', 'rb') as f:
    dat = pickle.load(f, encoding='bytes') 
print(dat.shape)
dat = dat.transpose(0,2,3,1)

print(dat.shape)
       
    
with open('chanlbl.p', 'rb') as f:
    lbl = pickle.load(f, encoding='bytes')
print(lbl.shape)
with open('datpointy.p', 'rb') as f:
    dat1 = pickle.load(f, encoding='bytes') 
print(dat1.shape)
dat1 = dat1.transpose(0,2,3,1)

print(dat1.shape)
       
    
with open('labelpointy.p', 'rb') as f:
    label1 = pickle.load(f, encoding='bytes')
print(label1.shape)
with open('datmudn.p', 'rb') as f:
    dat2 = pickle.load(f, encoding='bytes') 
print(dat2.shape)
dat2 = dat2.transpose(0,2,3,1)

print(dat2.shape)
       
    
with open('labelmudn.p', 'rb') as f:
    label2 = pickle.load(f, encoding='bytes')
print(label2.shape)

data1=np.concatenate((dat,dat1,dat2),axis=0)
label=np.concatenate((lbl,label1,label2),axis=0)
data=(data1-np.mean(data1))/np.std(data1)

print(data.shape)
print(label.shape)


training_data,eval_data,training_label,eval_label=train_test_split(data,label,test_size=0.005,random_state=42)
with open('dw.p', 'rb') as f:
    test = pickle.load(f, encoding='bytes') 
test=(test-np.mean(data1))/np.std(data1)
print(test.shape)


    

(375, 100, 156, 156)
(375, 156, 156, 100)
(375, 156, 156, 100)
(375, 100, 156, 156)
(375, 156, 156, 100)
(375, 156, 156, 100)
(375, 100, 156, 156)
(375, 156, 156, 100)
(375, 156, 156, 100)
(1125, 156, 156, 100)
(1125, 156, 156, 100)
(312, 312, 100)


In [7]:
training_label.shape



(1119, 156, 156, 100)

In [8]:
training_data.shape

(1119, 156, 156, 100)

In [9]:
eval_data.shape


(6, 156, 156, 100)

In [10]:
test=parse_image(test, 312, 312, 100)

In [11]:
test_data=test

In [12]:
def random_mini_batches(X, Y, mini_batch_size = 4):
    """
    Creates a list of random minibatches from (X, Y)
    
    Arguments:
    X -- input data, of shape (input size, number of examples) (m, Hi, Wi, Ci)
    Y -- true "label" vector (containing 0 if cat, 1 if non-cat), of shape (1, number of examples) (m, n_y)
    mini_batch_size - size of the mini-batches, integer
    seed -- this is only for the purpose of grading, so that you're "random minibatches are the same as ours.
    
    Returns:
    mini_batches -- list of synchronous (mini_batch_X, mini_batch_Y)
    """
    
    m = X.shape[0]                  # number of training examples
    mini_batches = []
   
    
    # Step 1: Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[permutation,:,:,:]
    shuffled_Y = Y[permutation,:,:,:]

    # Step 2: Partition (shuffled_X, shuffled_Y). Minus the end case.
    num_complete_minibatches = int(math.floor(m/mini_batch_size)) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[k * mini_batch_size : k * mini_batch_size + mini_batch_size,:,:,:]
        mini_batch_Y = shuffled_Y[k * mini_batch_size : k * mini_batch_size + mini_batch_size,:,:,:]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[num_complete_minibatches * mini_batch_size : m,:,:]
        mini_batch_Y = shuffled_Y[num_complete_minibatches * mini_batch_size : m,:,:]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    
    return mini_batches

In [13]:
test_data.shape

(4, 156, 156, 100)

In [14]:
def fast_hist(a, b, n):
    k = (a >= 0) & (a < n)
    return np.bincount(n * a[k].astype(int) + b[k], minlength=n**2).reshape(n, n)

In [15]:
def get_hist(predictions, labels):
    num_class = predictions.shape[4]
    batch_size = predictions.shape[0]
    hist = np.zeros((num_class, num_class))
    for i in range(batch_size):
        hist += fast_hist(labels[i].flatten(), predictions[i].argmax(3).flatten(), num_class)
    return hist

In [16]:
def MAX_VOTE(pred,prob,NUM_CLASS):
    """
    logit: the shape should be [NUM_SAMPLES,Batch_size, image_h,image_w,NUM_CLASS]
    pred: the shape should be[NUM_SAMPLES,NUM_PIXELS]
    label: the real label information for each image
    prob: the probability, the shape should be [NUM_SAMPLES,image_h,image_w,NUM_CLASS]
    Output:
    logit: which will feed into the Normal loss function to calculate loss and also accuracy!
    """

    image_h = 156
    image_w = 156
    image_d = 100
    NUM_SAMPLES = np.shape(pred)[0]
    #transpose the prediction to be [NUM_PIXELS,NUM_SAMPLES]
    pred_tot = np.transpose(pred)
    prob_re = np.reshape(prob,[NUM_SAMPLES,image_h*image_w*image_d,NUM_CLASS])
    prediction = []
    variance_final = []
    step = 0
    for i in pred_tot:
        
        value = np.bincount(i,minlength = NUM_CLASS)
        value_max = np.argmax(value)
        #indices = [k for k,j in enumerate(i) if j == value_max]
        indices = np.where(i == value_max)[0]
        prediction.append(value_max)
        variance_final.append(np.var(prob_re[indices,step,:],axis = 0))
        step = step+1
        
     
    return variance_final,prediction

In [17]:
def create_batches(photo, batch_size):
    
    while (True):
        for i in range(0,batch_size):
            yield(photo[i:i+batch_size]) 

In [18]:
global_step = tf.Variable(0, name="global_step", trainable=False)

with tf.name_scope("Input"):
    x = tf.placeholder(tf.float32, [None,156,156,100])
    y = tf.placeholder(tf.int32, [None,156,156,100])
    keepprob = tf.placeholder(tf.float32)
    is_training = tf.placeholder(tf.bool)
loss, logits = model5.inference(x,y, keepprob, is_training)

learning_rate = 0.05


pred = tf.cast(tf.argmax(tf.nn.softmax(logits), -1), tf.int32)



with tf.name_scope("Train"):
    grad_op = tf.train.AdamOptimizer(learning_rate)
    train_op = grad_op.minimize(loss, global_step=global_step)

tf.summary.scalar("cost", loss)
tf.summary.histogram('histogram_loss', loss)
summary_op = tf.summary.merge_all()
import os
try:
    os.mkdir('checkpoints')
except OSError:
    pass

try:
    os.mkdir('checkpoints/channel_01')
except OSError:
    pass
saver = tf.train.Saver()

init_op = tf.global_variables_initializer()


Tensor("Convnet1/convonet1/Relu:0", shape=(?, 156, 156, 100, 16), dtype=float32)
Tensor("Maxpool1/mpool1:0", shape=(?, 78, 78, 50, 16), dtype=float32)
Tensor("Convnet2/convonet2/Relu:0", shape=(?, 78, 78, 50, 16), dtype=float32)
Tensor("Maxpool2/mpool2:0", shape=(?, 39, 39, 25, 16), dtype=float32)
Tensor("Convnet3/convonet3/Relu:0", shape=(?, 39, 39, 25, 16), dtype=float32)
Tensor("Maxpool3/mpool3:0", shape=(?, 20, 20, 13, 16), dtype=float32)
Tensor("Convnet4/convonet4/Relu:0", shape=(?, 20, 20, 13, 16), dtype=float32)
Tensor("Maxpool4/mpool4:0", shape=(?, 10, 10, 7, 16), dtype=float32)
Tensor("Deconv4/upsamp4/conv3d_transpose:0", shape=(?, 20, 20, 13, 16), dtype=float32)
Tensor("Deconv4/conv_decode4/cond/Merge:0", shape=(?, 20, 20, 13, 16), dtype=float32)
Tensor("Deconv3/upsamp3/conv3d_transpose:0", shape=(?, 39, 39, 25, 16), dtype=float32)
Tensor("Deconv3/conv_decode3/cond/Merge:0", shape=(?, 39, 39, 25, 16), dtype=float32)
Tensor("Deconv2/upsamp2/conv3d_transpose:0", shape=(?, 78, 7

In [19]:
def draw_plots_bayes(images, labels, predicted_labels, uncertainty):
    
    num_images = len(images)
    
    cols = ['Input', 'Ground truth', 'Output', 'Uncertainty']
    rows = ['Image {}'.format(row) for row in range(1,num_images+1)]
    #rows = ['Worst', 'Average', 'Best']

    fig, axes = plt.subplots(nrows=num_images, ncols=4, figsize=(20,num_images*4))
    

    for i in range(num_images):

        plt.subplot(num_images, 4, (4*i+1))
        plt.imshow(images[i,:,:,50])
        #plt.ylabel("Image %d" % (i+1), size='18')
        plt.ylabel(rows[i], size='22')
        plt.xticks([])
        plt.yticks([])

        if (i==0): 
            plt.title(cols[0], size='22', va='bottom')

        plt.subplot(num_images, 4, (4*i+2))
        plt.imshow(labels[i,:,:,50])
        plt.xticks([])
        plt.yticks([])

        if (i==0): 
            plt.title(cols[1], size='22', va='bottom')

        plt.subplot(num_images, 4, (4*i+3))
        plt.imshow(predicted_labels[i,:,:,50])
        plt.xticks([])
        plt.yticks([])

        if (i==0): 
            plt.title(cols[2], size='22', va='bottom')
            
        plt.subplot(num_images, 4, (4*i+4))
        plt.imshow(uncertainty[i,:,:,50], cmap = 'Greys')
        plt.xticks([])
        plt.yticks([])

        if (i==0): 
            plt.title(cols[3], size='22', va='bottom')

    plt.show()

In [20]:
def draw_plots_bayes_external(images, predicted_labels, uncertainty):
    
    num_images = images.shape[0]
    
    cols = ['Input', 'Output', 'Uncertainty']
    rows = ['Image {}'.format(row) for row in range(1,num_images+1)]
    #rows = ['Worst', 'Average', 'Best']

    fig, axes = plt.subplots(nrows=num_images, ncols=3, figsize=(16,num_images*4))
    

    for i in range(num_images):

        plt.subplot(num_images, 3, (3*i+1))
        plt.imshow(images[i,:,:,50])
        #plt.ylabel("Image %d" % (i+1), size='18')
        plt.ylabel(rows[i], size='18')
        plt.xticks([])
        plt.yticks([])

        if (i==0): 
            plt.title(cols[0], size='18', va='bottom')

        plt.subplot(num_images, 3, (3*i+2))
        plt.imshow(predicted_labels[i,:,:,50])
        plt.xticks([])
        plt.yticks([])

        if (i==0): 
            plt.title(cols[1], size='18', va='bottom')
            
        plt.subplot(num_images, 3, (3*i+3))
        plt.imshow(uncertainty[i,:,:,50], cmap = 'Greys')
        plt.xticks([])
        plt.yticks([])

        if (i==0): 
            plt.title(cols[2], size='18', va='bottom')

plt.show()

In [21]:
def var_calculate(pred,prob_variance):
    """
    Inputs: 
    pred: predicted label, shape is [NUM_PIXEL,1]
    prob_variance: the total variance for 12 classes wrt each pixel, prob_variance shape [image_h,image_w,2]
    Output:
    var_one: corresponding variance in terms of the "optimal" label
    """
        
    image_h = 156
    image_w = 156
    image_c = 100
    NUM_CLASS = np.shape(prob_variance)[-1]
    var_sep = [] #var_sep is the corresponding variance if this pixel choose label k
    length_cur = 0 #length_cur represent how many pixels has been read for one images
    for row in np.reshape(prob_variance,[image_h*image_w*image_c,NUM_CLASS]):
        temp = row[pred[length_cur]]
        length_cur += 1
        var_sep.append(temp)
    var_one = np.reshape(var_sep,[image_h,image_w, image_c]) #var_one is the corresponding variance in terms of the "optimal" label
    
    return var_one

In [ ]:
begin_time = time.time()
num_epochs = 4
batch_size = 7
costs = []
accs=[]
import os
config = tf.ConfigProto()
config.gpu_options.allow_growth= True
sess = tf.Session(config=config)
init_op.run(session=sess)
# to visualize using TensorBoard
writer = tf.summary.FileWriter('./graphs/channel_01', sess.graph)
ckpt = tf.train.get_checkpoint_state(os.path.dirname('checkpoints/channel_01/checkpoint'))
# if that checkpoint exists, restore from checkpoint
if ckpt and ckpt.model_checkpoint_path:
    saver.restore(sess, ckpt.model_checkpoint_path)
output_list = None

start_time = time.time()
for epoch in range(num_epochs):
    minibatch_cost = 0.
    num_minibatches = int(training_data.shape[0] / batch_size) # number of minibatches of size minibatch_size in the train set
    
    minibatches = random_mini_batches(training_data, training_label, batch_size)
    
    count = 0
    i=0
    histo = np.zeros((2, 2))
    
    for minibatch in minibatches:

        # Select a minibatch
        (minibatch_X, minibatch_Y) = minibatch
        
        _, step, output, cost,summary = sess.run(
            [train_op, global_step, tf.nn.softmax(logits), loss,summary_op],
            feed_dict={x: minibatch_X, y: minibatch_Y, keepprob:0.7, is_training:True})
        count += 1
        
        hist = get_hist(output, minibatch_Y)
        acc = np.diag(hist).sum() / hist.sum()
        
            
        if count % 50 == 0 or i+1 == num_minibatches:
            elapsed_time = time.time() - start_time
            start_time = time.time()
            print("Step: %d, " % (step+1)+
                  "  Epoch: %2d," % (epoch+1)+
                  "  Batch: %3d of %3d," % (i+1, num_minibatches)+
                  "  Cost: %.4f," % cost+
                  "  Accuracy: %.4f," % acc+
                  " AvgTime/Sample: %3.2fms" % float(elapsed_time*1000/count/4))
            saver.save(sess, 'checkpoints/channel_01/channel-convnet', step)                  
            
            count = 0
        i+=1
            
        minibatch_cost += cost / num_minibatches
        histo += hist
        
    
    
    
    acc_total = np.diag(histo).sum() / histo.sum()
    iu = np.diag(histo) / (histo.sum(1) + histo.sum(0) - np.diag(histo))
    if epoch%1==0:
        print ("Cost after epoch %i: %f" % (epoch+1, minibatch_cost))
        print("acc: ", acc_total)
        print("mean IU: ", np.nanmean(iu))
    if epoch % 1 == 0:
        costs.append(minibatch_cost)
        accs.append(acc_total)
    
        
# plot the cost
plt.plot(np.squeeze(costs))
plt.ylabel('cost')
plt.xlabel('iterations')
plt.title("Learning rate =" + str(learning_rate))
plt.show()

plt.plot(np.squeeze(accs))
plt.ylabel('accuracy')
plt.xlabel('iterations')
plt.title("Learning rate =" + str(learning_rate))
plt.show()

print("Total Time: %3.2fs" % float(time.time() - begin_time))

#draw_plots_bayes(training_data[0:4,:,:,:], training_label[0:4,:,:,:], pred_tot[0:4,:,:,:], var_tot[0:4,:,:,:])


INFO:tensorflow:Restoring parameters from checkpoints/channel_01/channel-convnet-10404


In [ ]:
# plot the cost
plt.plot(np.squeeze(costs))
plt.ylabel('cost')
plt.xlabel('iterations')
plt.title("Learning rate =" + str(learning_rate))
plt.show()

plt.plot(np.squeeze(accs))
plt.ylabel('accuracy')
plt.xlabel('iterations')
plt.title("Learning rate =" + str(learning_rate))
plt.show()

print("Total Time: %3.2fs" % float(time.time() - begin_time))


In [ ]:
from IPython.core.pylabtools import figsize
import matplotlib.pyplot as plt
%matplotlib inline
figsize(50,50)

pred = np.argmax(output, axis=-1)

for select in range(1):
    plt.subplot(1,3, select*3+1)
    plt.imshow(np.transpose(minibatch_X[select, 100,:,:]))
    plt.subplot(1,3, select*3+2)
    plt.imshow(np.transpose(minibatch_Y[select, 100,:,:]))
    plt.subplot(1,3, select*3+3)
    plt.imshow(np.transpose(output[select, 100,:,:, 1]))

In [ ]:
eval_images = np.zeros([1, 156, 156, 100])
eval_lbl = np.zeros([1, 156, 156, 100])
null_labels = np.zeros_like(eval_images).astype(np.int32)
output_eval=None

for i in range(1):
    eval_images[i,:,:,:]=eval_data[i,:,:,:]
    eval_lbl[i,:,:,:]=eval_label[i,:,:,:]

num_sample_generate = 30
pred_tot = []
var_tot = []    

eval_output = sess.run(tf.nn.softmax(logits), feed_dict={x: eval_images, y: null_labels, keepprob:1, is_training:True})
if output_eval is not None:
    output_eval = np.concatenate((output_eval, eval_output), axis=0)
else:
    output_eval = eval_output

figsize(50,50)
for select in range(1):
    plt.subplot(4,3, select*3+1)
    plt.imshow(np.transpose(eval_images[select, :,:,69]))
    plt.subplot(4,3, select*3+2)
    plt.imshow(np.transpose(output_eval[select, :,:,69, 1]))
    plt.subplot(4,3, select*3+3)
    plt.imshow(np.transpose(eval_lbl[select, :,:,69]))


for image_batch, label_batch in zip(eval_images,eval_lbl):
    image_batch = np.reshape(image_batch,[1,156,156,100])
    label_batch = np.reshape(label_batch,[1,156,156,100])
    prob_iter_tot = []
    pred_iter_tot = []
    
    for iter_step in range(num_sample_generate):
        prob_iter_step = sess.run(tf.nn.softmax(logits), feed_dict = {x: image_batch, y: label_batch, keepprob:0.7, is_training:True}) 
        prob_iter_tot.append(prob_iter_step)
        pred_iter_tot.append(np.reshape(np.argmax(prob_iter_step,axis = -1),[-1]))
        
    prob_variance,pred=MAX_VOTE(pred_iter_tot,prob_iter_tot,2)
    acc_per=np.mean(np.equal(pred,np.reshape(label_batch,[-1])))
    var_one=var_calculate(pred,prob_variance)
    pred=np.reshape(pred,[156,156,100])
    print("Accuracy after 30 samples: %f" % (acc_per))
    #prob_mean = np.nanmean(prob_iter_tot,axis = 0)
    #prob_variance = np.var(prob_iter_tot, axis = 0)
    #pred = np.reshape(np.argmax(prob_mean,axis = -1),[-1]) #pred is the predicted label with the mean of generated samples
    #THIS TIME I DIDN'T INCLUDE TAU
    #var_one = var_calculate(pred,prob_variance)
    #pred = np.reshape(pred,[156,156,100])
    pred_tot.append(pred)
    var_tot.append(var_one)
pred_tot=np.array(pred_tot)
var_tot=np.array(var_tot)
draw_plots_bayes(eval_images, eval_lbl, pred_tot, var_tot)


In [ ]:
test_images = np.zeros([4, 156, 156, 100])

null_labels_test = np.zeros_like(test_images).astype(np.int32)
output_test=None
for i in range(4):
    test_images[i,:,:,:]=test_data[i,:,:,:]

num_sample_generate = 300
pred_tot = []
var_tot = []
prob_tot=[]

test_output = sess.run(tf.nn.softmax(logits), feed_dict={x: test_images, y: null_labels_test, keepprob:1, is_training:True})
if output_test is not None:
    output_test = np.concatenate((output_test, test_output), axis=0)
else:
    output_test = test_output

figsize(50,50)
for select in range(4):
    plt.subplot(2,4, select*2+1)
    plt.imshow(test_images[select, :,:,70])
    plt.subplot(2,4, select*2+2)
    plt.imshow(output_test[select,:,:,70, 1])

for image_batch, label_batch in zip(test_images,null_labels_test):
    image_batch = np.reshape(image_batch,[1,156,156,100])
    label_batch = np.reshape(label_batch,[1,156,156,100])
    prob_iter_tot = []
    pred_iter_tot = []
    
    for iter_step in range(num_sample_generate):
        prob_iter_step = sess.run(tf.nn.softmax(logits), feed_dict = {x: image_batch, y: label_batch, keepprob:0.7, is_training:True}) 
        prob_iter_tot.append(prob_iter_step)
        pred_iter_tot.append(np.reshape(np.argmax(prob_iter_step,axis = -1),[-1]))

    #prob_variance,pred=MAX_VOTE(pred_iter_tot,prob_iter_tot,2)
    #acc_per=np.mean(np.equal(pred,np.reshape(label_batch,[-1])))
    #var_one=var_calculate(pred,prob_variance)
    #pred=np.reshape(pred,[156,156,100])
    #print("Accuracy after 10 samples: %f" % (acc_per))
    
    prob_mean = np.nanmean(prob_iter_tot,axis = 0)
    prob_variance = np.var(prob_iter_tot, axis = 0)
    
    pred = np.reshape(np.argmax(prob_mean,axis = -1),[-1]) #pred is the predicted label with the mean of generated samples
    #THIS TIME I DIDN'T INCLUDE TAU
    var_one = var_calculate(pred,prob_variance)
    pred = np.reshape(pred,[156,156,100])
    prob=prob_mean[0,:,:,:,1]
    prob_variance=np.reshape(prob_variance,[156*156*100,2])
    variance=np.nanmean(prob_variance,axis=1)
    variance=np.reshape(variance,(156,156,100))
    pred_tot.append(prob)
    prob_tot.append(pred)
    var_tot.append(var_one)
pred_tot=np.array(pred_tot)
print(pred_tot.shape)
var_tot=np.array(var_tot)
draw_plots_bayes_external(test_images, pred_tot, var_tot)



In [ ]:
prob_tot=np.array(prob_tot)
prob_tot.shape

In [ ]:
out=np.concatenate((np.concatenate((pred_tot[0,:,:,:],pred_tot[1,:,:,:]),axis=1),np.concatenate((pred_tot[2,:,:,:],pred_tot[3,:,:,:]),axis=1)),axis=0)

In [ ]:
out=np.concatenate((np.concatenate((var_tot[0,:,:,:],var_tot[1,:,:,:]),axis=1),np.concatenate((var_tot[2,:,:,:],var_tot[3,:,:,:]),axis=1)),axis=0)

In [ ]:
out=np.concatenate((np.concatenate((output_test[0,:,:,:,1],output_test[1,:,:,:,1]),axis=1),np.concatenate((output_test[2,:,:,:,1],output_test[3,:,:,:,1]),axis=1)),axis=0)

In [ ]:
#out.tofile("channel156_01_6_var",format="%4")